# Samuel Chukwueze vs Ilias Akhomach - Radar Comparison
## Villarreal - Extremo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sqlalchemy import text
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
sys.path.insert(0, project_root)

from database.connection import get_db_manager

In [ ]:
MIN_MINUTES = 800
SEASON = '2223'

VILLARREAL_COLORS = {
    'primary': '#FFE500',
    'secondary': '#005187'
}

BARCELONA_COLORS = {
    'primary': '#004D98',
    'secondary': '#A50044'
}

EXTREMO_PLAYERS = [
    {'name': 'Samuel Chukwueze', 'search': 'Chukwueze', 'pos': 'FW', 'team': 'Villarreal'},
    {'name': 'Ilias Akhomach', 'search': 'Akhomach', 'pos': 'FW', 'team': 'Barcelona'},
]

SAVE_PATH_BASE = "./chukwueze_akhomach_radars/"
os.makedirs(SAVE_PATH_BASE, exist_ok=True)

In [ ]:
db = get_db_manager()

big5_leagues = ['ENG-Premier League', 'ESP-La Liga', 'ITA-Serie A', 'GER-Bundesliga', 'FRA-Ligue 1']
league_list_str = "', '".join(big5_leagues)

query = text(f"""
SELECT 
    unique_player_id,
    player_name,
    team,
    league,
    season,
    position,
    nationality,
    age,
    fbref_metrics,
    understat_metrics
FROM footballdecoded.players_domestic p
WHERE p.league IN ('{league_list_str}')
AND p.season = '{SEASON}'
ORDER BY p.league, p.season DESC, p.team, p.player_name
""")

df_raw = pd.read_sql(query, db.engine)
db.close()

print(f"Extraídos: {df_raw.shape[0]} registros")

In [ ]:
df_filtered = df_raw[
    (df_raw['fbref_metrics'].apply(
        lambda x: x.get('minutes_played', 0) if x else 0
    ) >= MIN_MINUTES) &
    (df_raw['position'].str.contains('FW', case=False, na=False))
].copy()

print(f"Filtro {MIN_MINUTES}+ min + FW: {df_filtered.shape[0]} jugadores")

In [ ]:
def extract_metrics(df, col_name):
    result = pd.DataFrame(index=df.index)
    all_keys = set()
    for _, row in df.iterrows():
        if isinstance(row[col_name], dict):
            all_keys.update(row[col_name].keys())
    
    for key in all_keys:
        values = []
        for _, row in df.iterrows():
            if isinstance(row[col_name], dict) and key in row[col_name]:
                raw_value = row[col_name][key]
                converted_value = _convert_to_float(raw_value)
                values.append(converted_value)
            else:
                values.append(np.nan)
        
        valid_count = pd.Series(values).notna().sum()
        if valid_count >= 5:
            result[key] = values
    return result

def _convert_to_float(value):
    if isinstance(value, (int, float)):
        return float(value)
    if value is None or pd.isna(value):
        return np.nan
    if isinstance(value, str):
        if value.strip() == '' or value.lower().strip() in ['nan', 'none', 'null', '-']:
            return np.nan
        try:
            return float(value)
        except (ValueError, TypeError):
            return np.nan
    return np.nan

fbref_nums = extract_metrics(df_filtered, 'fbref_metrics')
understat_nums = extract_metrics(df_filtered, 'understat_metrics')
print(f"FBref: {fbref_nums.shape[1]} métricas | Understat: {understat_nums.shape[1]} métricas")

In [ ]:
exclude_normalization = {
    'minutes_played', 'age', 'birth_year', 'games_started', 'minutes_per_game',
    'minutes_per_start', 'games', 'games_subs', 'unused_sub', 'points_per_game',
    'on_goals_for', 'on_goals_against', 'plus_minus', 'plus_minus_per90',
    'plus_minus_wowy', 'on_xg_for', 'on_xg_against', 'xg_plus_minus',
    'xg_plus_minus_per90', 'xg_plus_minus_wowy',
    'pass_completion_pct', 'shots_on_target_pct', 'Take-Ons_Succ%', 'Take-Ons_Tkld%', 
    'Aerial Duels_Won%', 'Challenges_Tkl%', 'Save%', 'Launched_Cmp%', 'Crosses_Stp%',
    'shots_per_90', 'GA90', 'GCA_GCA90', 'SCA_SCA90', 'Team Success_+/-90', 'SoT/90',
    'npxG/Sh', 'xG+xAG', 'non_penalty_xG_plus_xAG', 'avg_shot_distance', 'minutes_per_match',
    'Passes_AvgLen', 'Goal Kicks_AvgLen', 'Starts_Mn/Start', 'Subs_Mn/Sub', 'Min%',
    'matches_played', 'matches_started', 'wins', 'draws', 'losses',
    'understat_buildup_involvement_pct', 'understat_player_id', 'understat_team_id', 
    'CS%', 'CS', 'PSxG+/-', 'PSxG', 'Save_Save%',
    'Sweeper_#OPA', 'Sweeper_#OPA/90', 'Sweeper_AvgDist', 'Goal Kicks_Launch%', 'G-xG'
}

fbref_per90 = fbref_nums.loc[:, ~fbref_nums.columns.isin(exclude_normalization)]
fbref_per90 = (fbref_per90.div(fbref_nums['minutes_played'], axis=0) * 90).round(3)
fbref_per90.columns = [f'{col}_per90' for col in fbref_per90.columns]

understat_per90 = understat_nums.loc[:, ~understat_nums.columns.isin(exclude_normalization)]
understat_per90 = (understat_per90.div(fbref_nums['minutes_played'], axis=0) * 90).round(3)
understat_per90.columns = [f'{col}_per90' for col in understat_per90.columns]

print(f"Per90: {fbref_per90.shape[1]} + {understat_per90.shape[1]}")

all_metrics = pd.concat([fbref_nums, understat_nums, fbref_per90, understat_per90], axis=1)

In [ ]:
all_percentiles = all_metrics.rank(pct=True) * 98 + 1
all_percentiles = all_percentiles.round(0).astype('Int64')
all_percentiles.columns = [f'{col}_pct' for col in all_percentiles.columns]

print(f"Métricas totales: {all_metrics.shape[1]}")
print(f"Percentiles totales: {all_percentiles.shape[1]}")

In [ ]:
base_cols = ['unique_player_id', 'player_name', 'team', 'league', 'season', 'position']

df_final = pd.concat([
    df_filtered[base_cols],
    all_metrics,
    all_percentiles
], axis=1)

def clean_series_values(df):
    for col in df.columns:
        series_mask = df[col].apply(lambda x: isinstance(x, pd.Series))
        if series_mask.any():
            df[col] = df[col].apply(
                lambda x: x.iloc[0] if isinstance(x, pd.Series) and not x.empty else x
            )
    return df

df_final = clean_series_values(df_final)
problem_cols = ['shots_on_target_pct']
df_final = df_final.drop(columns=[col for col in problem_cols if col in df_final.columns])

print(f"DataFrame final: {df_final.shape[0]} filas, {df_final.shape[1]} columnas")

In [ ]:
player_ids = {}

for player_config in EXTREMO_PLAYERS:
    player_name = player_config['name']
    search_pattern = player_config['search']
    
    search_results = df_filtered[
        df_filtered['player_name'].str.contains(search_pattern, case=False, na=False)
    ]
    
    if len(search_results) > 0:
        print(f"\n{player_name}:")
        for _, p in search_results.iterrows():
            minutes = p['fbref_metrics'].get('minutes_played', 0) if isinstance(p['fbref_metrics'], dict) else 0
            print(f"  {p['player_name']} ({p['team']}) - {minutes} min")
            
            if player_name not in player_ids:
                player_ids[player_name] = p['unique_player_id']
                break

print(f"\n✓ Jugadores encontrados: {len(player_ids)}")

In [ ]:
extremo_metrics = [
    'expected_goals_per90', 'shots_per90', 'expected_assists_per90', 'Take-Ons_Succ_per90',
    'Touches_Att Pen_per90', 'SCA_SCA90', 'Crs_per90', 'Carries_PrgC_per90',
    'npxG/Sh', 'Fld_per90'
]
extremo_titles = [
    'Expected Goals', 'Shots', 'Expected Assists', 'Dribbles',
    'Touches in Box', 'Shot Creating Actions', 'Crosses', 'Progressive Carries',
    'npxG per Shot', 'Fouls Drawn'
]

METRICS_BY_POSITION = {
    'extremo': {'metrics': extremo_metrics, 'titles': extremo_titles}
}

print(f"Métricas extremo: {len(extremo_metrics)}")

In [ ]:
COMPARISON = {
    'name': 'Chukwueze_vs_Akhomach',
    'player1': 'Samuel Chukwueze',
    'player2': 'Ilias Akhomach', 
    'position_type': 'EXTREMO',
    'metrics_key': 'extremo',
    'title': 'Samuel Chukwueze vs Ilias Akhomach (22/23)',
    'faces': ['/home/jaime/FD/data/blog/caras/tfm/chukwueze.png', '/home/jaime/FD/data/blog/caras/tfm/akhomach.png'],
    'colors': [VILLARREAL_COLORS['primary'], BARCELONA_COLORS['primary']]
}

In [ ]:
from viz.swarm_radar import create_player_radar
from viz.stats_radar import create_stats_table, combine_radar_and_table
from IPython.display import Image as IPImage, display

comparison = COMPARISON
config_name = comparison['name']
player1_name = comparison['player1'] 
player2_name = comparison['player2']
title = comparison['title']
faces = comparison['faces']
colors = comparison['colors']
metrics_key = comparison['metrics_key']

player1_id = player_ids[player1_name]
player2_id = player_ids[player2_name]

metrics_config = METRICS_BY_POSITION[metrics_key]
metrics = metrics_config['metrics']
metric_titles = metrics_config['titles']

position_df = df_final[df_final['position'].str.contains('FW', case=False, na=False)].copy()
position_filter = "FW"

player1_team = df_final.loc[df_final['unique_player_id'] == player1_id, 'team'].iloc[0]
player2_team = df_final.loc[df_final['unique_player_id'] == player2_id, 'team'].iloc[0]

team_colors = colors
team_logos = {}
if faces[0]:
    team_logos[player1_team] = faces[0]
if len(faces) > 1 and faces[1]:
    team_logos[player2_team] = faces[1]

try:
    create_player_radar(
        df_data=position_df,
        player_1_id=player1_id,
        player_2_id=player2_id,
        metrics=metrics,
        metric_titles=metric_titles,
        team_colors=team_colors,
        save_path=f"{SAVE_PATH_BASE}{config_name}_radar.png",
        use_swarm=False,
        show_plot=False
    )
    
    create_stats_table(
        df_data=position_df,
        player_1_id=player1_id,
        player_2_id=player2_id,
        metrics=metrics,
        metric_titles=metric_titles,
        team_colors=team_colors,
        team_logos=team_logos,
        save_path=f"{SAVE_PATH_BASE}{config_name}_table.png",
        footer_text=f"Stats (per 90 mins) & Percentiles\nvs Top-5 League {position_filter} ({MIN_MINUTES}+ mins) 22/23",
        show_plot=False
    )
    
    combine_radar_and_table(
        radar_path=f"{SAVE_PATH_BASE}{config_name}_radar.png",
        table_path=f"{SAVE_PATH_BASE}{config_name}_table.png", 
        output_path=f"{SAVE_PATH_BASE}{config_name}_combined.png"
    )
    
    print(f"✓ Completed: {config_name}_combined.png")
    display(IPImage(f"{SAVE_PATH_BASE}{config_name}_combined.png"))
    
except Exception as e:
    print(f"✗ Error: {str(e)}")
    import traceback
    traceback.print_exc()